In [1]:
import pandas as pd
import efficientnet.keras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
poster_df = pd.read_csv("../data/MovieGenre.csv", encoding = "ISO-8859-1")
poster_df = poster_df.drop_duplicates(subset=['imdbId'], keep="last")

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255., rotation_range=40, width_shift_range=0.2, 
                                   height_shift_range=0.2, horizontal_flip=True, validation_split=0.25)

train_generator = train_datagen.flow_from_directory(
        directory='../data/posters',
        subset="training",
        batch_size=32,
        class_mode='categorical')

valid_generator = train_datagen.flow_from_directory(
        directory='../data/posters',
        subset="validation",
        batch_size=32,
        class_mode='categorical')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in model.layers:
    layer.trainable = False
    
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(input = model.input, output = predictions)

model_final.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy',recall,precision])

model_final.fit(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)

Reference:
1. https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/